In [7]:
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

model_name = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

def load_and_preprocess_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)

    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

    return waveform.squeeze().numpy()

def transcribe_audio(audio_path):
    input_audio = load_and_preprocess_audio(audio_path)

    input_values = processor(input_audio, sampling_rate=16000, return_tensors="pt").input_values

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription[0]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
audio_file = "data/test-1.wav"
transcription = transcribe_audio(audio_file)
print(transcription)

HELLO MY NAME IS NATANIA
